In [24]:
from astropy.io import fits 
import matplotlib.pyplot as plt
import numpy as np 
from scipy.optimize import minimize, rosen, rosen_der
from scipy import interpolate
import scipy
import os 
from scipy.optimize import curve_fit
from scipy.integrate import simps
import math
from astropy.io import ascii
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "text.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
plt.rcParams.update(params)
from astropy.cosmology import WMAP9 as cosmo

In [26]:
stuff = fits.open("C:/Users/19133/Documents/Research/Kirkpatrick/S82X_catalog_w_mbh_Cooke_Ricci.fits")

In [27]:
stuff[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  646 / length of dimension 1                          
NAXIS2  =                 6181 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  119 / number of table fields                         
TTYPE1  = 'MSID    '                                                            
TFORM1  = 'J       '                                                            
TTYPE2  = 'REC_NO  '                                                            
TFORM2  = 'J       '                                                            
TTYPE3  = 'OBSID   '        

In [34]:
def mag2flux(m):  
    l = np.zeros(len(m))
    
    for i in range(len(m)): 
        if m[i] <= -99: 
            l[i] = np.nan #when doing things like mathematical processes nan overrides anything
            #and just makes the end value nan
            #this is convenient because it invalidates any process attempted with a non detected
            #and matplotlib just doesn't plot nan values
            #so they're easily skipped 
            
        else: 
            power = m[i]/(-5/2)
            l[i] = (10**power)*3631*1000
    return l 

In [33]:
def mag2flux_err(m_err,flux): 
    l = np.zeros(len(m_err))
    
    for i in range(len(m_err)): 
        if m_err[i] <= -99: 
            l[i] = np.nan 
        
        else: 
            l[i] = m_err[i]*flux[i]*np.log(10)
    return l 

In [35]:
#check that this is right?? 
def xray_to_mjy(m,kev):
    freq = kev*(2.41799*(10**(17)))
    l = np.zeros(len(m))
    for i in range(len(m)): 
        if m[i] == 0: 
            l[i] = np.nan #when doing things like mathematical processes nan overrides anything
            #and just makes the end value nan
            #this is convenient because it invalidates any process attempted with a non detected
            #and matplotlib just doesn't plot nan values
            #so they're easily skipped 
        else: 
            mjy = m[i]*1000/(freq*10**(-23))
            l[i] = mjy
    return l 

In [38]:
#xray 
#hard - 2-10 kev, xray_boxcar_2to10keV
#soft - 0.5-2 kev, xray_boxcar_0p5to2keV
#already in flux (mjys though?)
X_hard = xray_to_mjy(stuff[1].data['HARD_FLUX'],5)
X_hard_err = xray_to_mjy(stuff[1].data['HARD_FLUX_ERROR'],5)
X_soft = xray_to_mjy(stuff[1].data['SOFT_FLUX'],1)
X_soft_err = xray_to_mjy(stuff[1].data['SOFT_FLUX_ERROR'],1)

#fuv, nuv
#galex for filters 
#FUV.dat, NUV.dat
#mags
FUV = mag2flux(stuff[1].data['MAG_FUV'])
FUV_err = mag2flux_err(stuff[1].data['MAGERR_FUV'],FUV)
NUV = mag2flux(stuff[1].data['MAG_NUV'])
NUV_err = mag2flux_err(stuff[1].data['MAGERR_NUV'],NUV)

#ugriz 
#prime for filters 
#mags
U = mag2flux(stuff[1].data['U'])
G = mag2flux(stuff[1].data['G'])
R = mag2flux(stuff[1].data['R'])
I = mag2flux(stuff[1].data['I'])
Z = mag2flux(stuff[1].data['Z'])

U_err = mag2flux_err(stuff[1].data['U_ERR'],U)
G_err = mag2flux_err(stuff[1].data['G_ERR'],G)
R_err = mag2flux_err(stuff[1].data['R_ERR'],R)
I_err = mag2flux_err(stuff[1].data['I_ERR'],I)
Z_err = mag2flux_err(stuff[1].data['Z_ERR'],Z)

#JHK
#use vista for VHS 
#use ukirt for UK 
#mags
#J,H,K
JVHS = mag2flux(stuff[1].data['JVHS'])
JVHS_err = mag2flux_err(stuff[1].data['JVHS_ERR'],JVHS)
HVHS = mag2flux(stuff[1].data['HVHS'])
HVHS_err = mag2flux_err(stuff[1].data['HVHS_ERR'],HVHS)
KVHS = mag2flux(stuff[1].data['KVHS'])
KVHS_err = mag2flux_err(stuff[1].data['KVHS_ERR'],KVHS)

JUK = mag2flux(stuff[1].data['JUK'])
JUK_err = mag2flux_err(stuff[1].data['JUK_ERR'],JUK)
HUK = mag2flux(stuff[1].data['HUK'])
HUK_err = mag2flux_err(stuff[1].data['HUK_ERR'],HUK)
KUK = mag2flux(stuff[1].data['KUK'])
KUK_err = mag2flux_err(stuff[1].data['KUK_ERR'],KUK)

#redshift 
spec_z = stuff[1].data['SPEC_Z']
photo_z = stuff[1].data['PHOTO_Z']

#CH1, CH2 
#both in data use IRAC, just different papers 
#so it probably doesn't make a ton of difference 
#so I'm just gonna use spies for now 
#mags
#IRAC1. IRAC2
CH1 = mag2flux(stuff[1].data['CH1_SPIES'])
CH1_err = mag2flux_err(stuff[1].data['CH1_SPIES_ERR'],CH1)
CH2 = mag2flux(stuff[1].data['CH2_SPIES'])
CH2_err = mag2flux_err(stuff[1].data['CH2_SPIES_ERR'],CH2)

#W1, W2, W3, W4
#allwise for filters 
#mags 
#none - -999
#WISE1, WISE2, WISE3, WISE4 
W1 = mag2flux(stuff[1].data['W1'])
W1_err = mag2flux_err(stuff[1].data['W1_ERR'],W1)
W2 = mag2flux(stuff[1].data['W2'])
W2_err = mag2flux_err(stuff[1].data['W2_ERR'],W2)
W3 = mag2flux(stuff[1].data['W3'])
W3_err = mag2flux_err(stuff[1].data['W3_ERR'],W3)
W4 = mag2flux(stuff[1].data['W4'])
W4_err = mag2flux_err(stuff[1].data['W4_ERR'],W4)

In [39]:
#data has two id columns where it changes which one it used 
#so I'm just consolidating them into one list of unique ids for each source 
msids = stuff[1].data['MSID']
rec_nos = stuff[1].data['REC_NO']
msids[np.where(stuff[1].data['MSID'] == 0)] = rec_nos[np.where(stuff[1].data['MSID'] == 0)]
ids = msids

In [40]:
#raw data 

fluxes = np.array([X_hard,X_soft,FUV,NUV,U,G,R,I,Z,JUK,HUK,KUK,W1,CH1,CH2,W2,W3,W4])
errors = np.array([X_hard_err,X_soft_err,FUV_err,NUV_err,
                   U_err,G_err,R_err,I_err,Z_err,JUK_err,HUK_err,KUK_err,
                   W1_err,CH1_err,CH2_err,W2_err,W3_err,W4_err])

In [41]:
#file comes with two redshift columns 
#spec is prefereable, so if we have it, use that
#otherwise use photo

spec_z = stuff[1].data['SPEC_Z']
photo_z = stuff[1].data['PHOTO_Z']

#they denote no detection in a couple ways, just turn them all to nans 
spec_z[np.where(spec_z <= -99)[0]] = np.nan
spec_z[np.where(spec_z == 0)[0]] = np.nan

#starting with spec
real_z = spec_z

#if spec has no detection, either use the photo or input nan if no photo as well 
for i in range(len(real_z)):
    if np.isnan(real_z[i]) == True: 
        if photo_z[i] <= -99:
            real_z[i] = np.nan
        else:
            real_z[i] = photo_z[i]
    else:
        True 

In [42]:
#removing sources with less than 2 bands detected because you can't fit with that 

#turning all nans to zeros so I can use np count nonzero function
fluxes[np.where(np.isnan(fluxes) == True)] = 0

indeces = [] 

#for each source, looking at its individual list of band fluxes 
#if there are less than two detections, save this index as one to get rid of 
for x in range(len(fluxes[0])):
    a = np.count_nonzero(fluxes[:,x])
    if a <= 1:
        indeces.append(x)
    #also taking away indeces that have galaxies without a redshift
    elif (np.isnan(real_z[x]) == True):
        indeces.append(x)
    else: 
        True 

#turning zeros back into nans to make math calculations work smoother later         
fluxes[np.where(fluxes == 0)] = np.nan
errors[np.where(errors == 0)] = np.nan

fluxes = np.delete(fluxes,indeces,1)
real_z = np.delete(real_z,indeces)
errors = np.delete(errors,indeces,1)
ids = np.delete(ids,indeces)

In [45]:
from prettytable import PrettyTable
#putting data to a file for XCigale 

x = PrettyTable()

x.add_column("# id", ids)
x.add_column("redshift", real_z)
x.add_column("xray_boxcar_2to10keV",fluxes[0])
x.add_column("xray_boxcar_2to10keV_err",errors[0])
x.add_column("xray_boxcar_0p5to2keV",fluxes[1])
x.add_column("xray_boxcar_0p5to2keV_err",errors[1])
x.add_column("FUV",fluxes[2])
x.add_column("FUV_err",errors[2])
x.add_column("NUV",fluxes[3])
x.add_column("NUV_err",errors[3])
x.add_column("u_prime",fluxes[4])
x.add_column("u_prime_err",errors[4])
x.add_column("g_prime",fluxes[5])
x.add_column("g_prime_err",errors[5])
x.add_column("r_prime",fluxes[6])
x.add_column("r_prime_err",errors[6])
x.add_column("i_prime",fluxes[7])
x.add_column("i_prime_err",errors[7])
x.add_column("z_prime",fluxes[8])
x.add_column("z_prime_err",errors[8])
x.add_column("J",fluxes[9])
x.add_column("J_err",errors[9])
x.add_column("H",fluxes[10])
x.add_column("H_err",errors[10])
x.add_column("K",fluxes[11])
x.add_column("K_err",errors[11])
x.add_column("WISE1",fluxes[12])
x.add_column("WISE1_err",errors[12])
x.add_column("IRAC1",fluxes[13])
x.add_column("IRAC1_err",errors[13])
x.add_column("IRAC2",fluxes[14])
x.add_column("IRAC2_err",errors[14])
x.add_column("WISE2",fluxes[15])
x.add_column("WISE2_err",errors[15])
x.add_column("WISE3",fluxes[16])
x.add_column("WISE3_err",errors[16])
x.add_column("WISE4",fluxes[17])
x.add_column("WISE4_err",errors[17])

x.border=False

In [46]:
#writing to file 
cigale_data = open("C:/Users/19133/Documents/cigale-xray/cigale-xray/pcigale/xcigale_data_04_06.txt", "w+")
cigale_data.close()
with open("C:/Users/19133/Documents/cigale-xray/cigale-xray/pcigale/xcigale_data_04_06.txt",'w') as f: 
    f.write(x.get_string())